If you use this code, please cite

[Kazuki Nakajima, Takeaki Uno. Inference and Visualization of Community Structure in Attributed Hypergraphs Using Mixed-Membership Stochastic Block Models. Social Network Analysis and Mining. Vol. 15, Article No. 5 (2025).](https://doi.org/10.1007/s13278-025-01440-z)

In [ ]:
import time
import numpy as np
import hypergraph, hymmsbm, hycosbm, comm_vis
import matplotlib.pyplot as plt
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_mutual_info_score
import math
from scipy import stats
import seaborn as sns
import random
from itertools import combinations
from scipy.stats import wilcoxon
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

fontsize = 40
datadir = "./data"

plt.rcParams["font.size"] = fontsize
plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.rcParams['xtick.major.width'] = 2.5 
plt.rcParams['xtick.minor.width'] = 1 
plt.rcParams['ytick.major.width'] = 2.5 
plt.rcParams['ytick.minor.width'] = 1 
plt.rcParams['axes.linewidth'] = 2.5 
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['xtick.minor.size'] = 12.5
plt.rcParams['ytick.major.size'] = 15
plt.rcParams['ytick.minor.size'] = 12.5
plt.rcParams['hatch.linewidth'] = 0.3

random_state = 42

In [ ]:
import os

if not os.path.exists("./figs"):
    os.makedirs("./figs")

In [ ]:
# Fig.2
N = 1000
beta_ = 1
k = 3
K = 2
alpha_lst = [1, 10]
p_att_lst = [0.5]
label_name = ["First community","Second community"]
label_order = [0, 1]

for p_att in p_att_lst:
    for alpha in alpha_lst:
        
        data_name = 'syn_hard_' + str(N) + '_' + str(p_att) + '_' + str(alpha)
        G, community = hypergraph.generate_uniform_hsbm(N, k, K, alpha, beta_, p_att)
        
        if p_att == 0.5:
            model_name = "hymmsbm"
            model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
            best_loglik, (U, W) = model.fit()
        else:
            model_name = "hycosbm"
            gamma = 2 * p_att - 1
            model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
            best_loglik, (U, W, beta) = model.fit()
        
        G.X = community
        
        for dim_red in ['t-SNE', 'UMAP', 'triMAP', 'PaCMAP']:
            comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red=dim_red, fig_show=True)

In [ ]:
# Fig.3
N = 1000
D = 3
K = 2
p_lst = [1.0, 0.8]
w_in_lst = [0.1]
label_name = ["Group 1","Group 2"]
label_order = [0, 1]

for p in p_lst:
    for w_in in w_in_lst:
        data_name = 'syn_mix_' + str(N) + '_' + str(w_in) + '_' + str(p)
        
        G, community = hypergraph.generate_hymmsbm(N, D, K, p, w_in)
        
        model_name = "hymmsbm"
        model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
        best_loglik, (U, W) = model.fit()
        
        G.X = community
        
        for dim_red in ['t-SNE', 'UMAP', 'triMAP', 'PaCMAP']:
                comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red=dim_red, fig_show=True)

In [ ]:
# Fig.4
N = 1000
beta_ = 1
k = 3
K = 2
alpha_lst = [4]
p_att_lst = [0.6, 0.9]
label_name = ["First community","Second community"]
label_order = [0, 1]

for p_att in p_att_lst:
    for alpha in alpha_lst:
        
        data_name = 'syn_hard_' + str(N) + '_' + str(p_att) + '_' + str(alpha)
        G, community = hypergraph.generate_uniform_hsbm(N, k, K, alpha, beta_, p_att)
        
        model_name = "hycosbm"
        gamma = 2 * p_att - 1
        model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
        best_loglik, (U, W, beta) = model.fit()
        
        G.X = community
        
        for dim_red in ['t-SNE', 'UMAP', 'triMAP', 'PaCMAP']:
            comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red=dim_red, fig_show=True)

In [ ]:
def hymmsbm_hyperparameter_tuning(G):
    t_s = time.time()
    best_auc, K, auc_score_ = hymmsbm.HyperParamTuning(G).run(G)
    t_e = time.time()
    print("HyMMSBM", "Best hyperparameter set", best_auc, K)
    print("Elapsed time", t_e - t_s)
    
    auc_score = {}
    for i in range(0, auc_score_.shape[0]):
        K = hymmsbm.HyperParamTuning(G).K_lst[i]
        auc_score[K] = []
        for j in range(0, auc_score_.shape[1]):
            auc_score[K].append(auc_score_[i, j])

    return auc_score

In [ ]:
def hycosbm_hyperparamter_tuning(G):
    t_s = time.time()
    best_auc, (K, gamma), auc_score_ = hycosbm.HyperParamTuning(G).run(G)
    t_e = time.time()
    print("HyCoSBM", "Best hyperparameter set", best_auc, K, gamma)
    print("Elapsed time", t_e - t_s)

    auc_score = {}
    for i in range(0, auc_score_.shape[0]):
        K = hycosbm.HyperParamTuning(G).K_lst[i]
        for j in range(0, auc_score_.shape[1]):
            gamma = hycosbm.HyperParamTuning(G).gamma_lst[j]
            auc_score[(K, gamma)] = []
            for k in range(0, auc_score_.shape[2]):
                auc_score[(K, gamma)].append(auc_score_[i, j, k])

    return auc_score

In [ ]:
def construct_train_and_test_sets(G, p=0.2):
    hyperedge_indices = list(range(0, G.M))
    random.shuffle(hyperedge_indices)

    E_train, E_test = [G.E[hyperedge_indices[i]] for i in range(0, int((1 - p) * G.M))], [G.E[hyperedge_indices[i]] for i in range(int((1 - p) * G.M), G.M)]
    A_train, A_test = [G.A[hyperedge_indices[i]] for i in range(0, int((1 - p) * G.M))], [G.A[hyperedge_indices[i]] for i in range(int((1 - p) * G.M), G.M)]

    G_train = hypergraph.HyperGraph(G.N, int(len(E_train)), G.Z)
    G_train.E = E_train
    G_train.A = A_train
    G_train.X = G.X

    G_test = hypergraph.HyperGraph(G.N, int(len(E_test)), G.Z)
    G_test.E = E_test
    G_test.A = A_test
    G_test.X = G.X

    return G_train, G_test

In [ ]:
def construct_R_lst(G_test):
    R = []
    sampled_edges = []
    for m in range(0, G_test.M):
        e, e_ = set(G_test.E[m]), set()
        s = len(e)
        flag = True
        while flag:
            e_ = set(random.sample(range(G_test.N), k=s))
            if len(e_) == s and e_ not in G.E and e_ not in sampled_edges:
                flag = False
                sampled_edges.append(e_)
        R.append((e, e_))
        
    return R

In [ ]:
def calc_auc(U, W, R):
    auc = 0.0
    for (e, e_) in R:
        param0 = sum([(U[i_] * W * U[j_].T).sum() for (i_, j_) in list(combinations(sorted(list(e_)), 2))])
        param1 = sum([(U[i_] * W * U[j_].T).sum() for (i_, j_) in list(combinations(sorted(list(e)), 2))])

        if param1 > param0:
            auc += 1.0
        elif math.isclose(param0, param1):
            auc += 0.5
    
    return float(auc) / len(R)

In [ ]:
def signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100):
    
    x, y = [], []
    for r in range(0, num_samples):
        G_train, G_test = construct_train_and_test_sets(G)
        R = construct_R_lst(G_test)

        K = hymmsbm_best_param
        model = hymmsbm.HyMMSBM(G_train, K)
        best_loglik, (U, W) = model.fit()
        auc = calc_auc(U, W, R)
        x.append(auc)
        
        (K, gamma) = hycosbm_best_param
        model = hycosbm.HyCoSBM(G_train, K, gamma)
        best_loglik, (U, W, Beta) = model.fit()
        auc = calc_auc(U, W, R)
        y.append(auc)
        
    print(x)
    print(y)
    
    return wilcoxon(x,y,alternative='less',method='exact')

In [ ]:
data_name = "workplace"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 5
hycosbm_best_param = (5, 0.9)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))

In [ ]:
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)

In [ ]:
label_name = ["DISQ","DMCT","DSE","SFLE","SRH"]
label_order = [0,1,2,3,4]

In [ ]:
# Hy-MMSBM
model_name = "hymmsbm"
K = 5
model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
best_loglik, (U, W) = model.fit()
community_order = [4,1,2,3,0]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')

In [ ]:
# HyCoSBM
model_name = "hycosbm"
K, gamma = 5, 0.9
model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
best_loglik, (U, W, beta) = model.fit()
community_order = [4,3,1,2,0]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.plot_inferred_attribute_correlation_matrix(data_name, beta, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')

In [ ]:
data_name = "hospital"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 2
hycosbm_best_param = (2, 0.3)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))

In [ ]:
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)

In [ ]:
label_name = ["ADM","MED","NUR","PAT"]
label_order = [0,1,2,3]

In [ ]:
# Hy-MMSBM
model_name = "hymmsbm"
K = 2
model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
best_loglik, (U, W) = model.fit()
community_order = [0, 1]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name)

In [ ]:
# HyCoSBM
model_name = "hycosbm"
K, gamma = 2, 0.7
model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
best_loglik, (U, W, beta) = model.fit()
community_order = [0, 1]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.plot_inferred_attribute_correlation_matrix(data_name, beta, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name)

In [ ]:
data_name = "contact-high-school"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 3
hycosbm_best_param = (9, 0.9)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))

In [ ]:
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)

In [ ]:
label_name = ["2BIO1", "2BIO2", "2BIO3", "MP*1", "MP*2", "PSI*", "PC", "PC*", "MP"]
label_order = [0, 1, 2, 8, 3, 4, 6, 7, 5]

In [ ]:
# Hy-MMSBM
model_name = "hymmsbm"
K = 3
model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
best_loglik, (U, W) = model.fit()
community_order = [0, 1, 2]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name, dim_red='t-SNE')

In [ ]:
# HyCoSBM
model_name = "hycosbm"
K, gamma = 9, 0.9
model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
best_loglik, (U, W, beta) = model.fit()
community_order = [1,4,5,6,3,8,2,0,7]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.plot_inferred_attribute_correlation_matrix(data_name, beta, label_name, label_order, community_order, model_name)
comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name, dim_red='t-SNE')
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name, dim_red='t-SNE')

In [ ]:
data_name = "contact-primary-school"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 3
hycosbm_best_param = (11, 0.8)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)

In [ ]:
label_name = ["5B", "5A", "4A", "Teachers", "3B", "4B", "2A", "1B", "2B", "1A", "3A"]
label_order = [9, 7, 6, 8, 10, 4, 2, 5, 1, 0, 3]

In [ ]:
# Hy-MMSBM
model_name = "hymmsbm"
K = 3
model = hymmsbm.HyMMSBM(G, K, random_state=random_state)
best_loglik, (U, W) = model.fit()
community_order = [0, 2, 1]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name)
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name)

In [ ]:
# HyCoSBM
model_name = "hycosbm"
K, gamma = 11, 0.8
model = hycosbm.HyCoSBM(G, K, gamma, random_state=random_state)
best_loglik, (U, W, beta) = model.fit()
community_order = [9,0,3,4,8,6,10,1,2,7,5]
comm_vis.plot_inferred_membership_matrix(G, data_name, U, label_name, label_order, community_order, model_name)
comm_vis.plot_inferred_attribute_correlation_matrix(data_name, beta, label_name, label_order, community_order, model_name)
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "euclidean", model_name)
#comm_vis.node_embedding(G, data_name, U, W, label_name, label_order, random_state, "cosine", model_name)

In [ ]:
data_name = "house-committees"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 15
hycosbm_best_param = (4, 0.4)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)

In [ ]:
data_name = "senate-committees"
G = hypergraph.read_empirical_hypergraph_data(data_name, print_info=True)

In [ ]:
hymmsbm_auc_score = hymmsbm_hyperparameter_tuning(G)

In [ ]:
hycosbm_auc_score = hycosbm_hyperparamter_tuning(G)

In [ ]:
hymmsbm_best_param = 3
hycosbm_best_param = (4, 0.7)
#print(stats.ttest_ind(list(hymmsbm_auc_score[hymmsbm_best_param]), hycosbm_auc_score[hycosbm_best_param], equal_var=False, alternative='less'))
signed_rank_test(G, hymmsbm_best_param, hycosbm_best_param, num_samples=100)